In [25]:
import numpy as np
import keras
import cv2
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
labels_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv'

In [3]:
dft = pd.read_csv(labels_path, skiprows=lambda x: x in range(1,5000,5))
dft["smiling"] = dft["smiling"].replace(to_replace=[-1], value=['Frown'])
dft["smiling"] = dft["smiling"].replace(to_replace=[1], value=['Smile'])
dft["gender"] = dft["gender"].replace(to_replace=[-1], value=['Female'])
dft["gender"] = dft["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dft["smiling"])
one_hot_g = pd.get_dummies(dft["gender"])
dft = dft.drop(columns=["gender", "smiling"])
dft = dft.join(one_hot_g)
dft = dft.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(
    dataframe=dft, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Frown", "Smile"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 4000 validated image filenames.


In [5]:
dfv = pd.read_csv(labels_path, skiprows=lambda x: x not in range(0,5000,5))
dfv["smiling"] = dfv["smiling"].replace(to_replace=[-1], value=['Frown'])
dfv["smiling"] = dfv["smiling"].replace(to_replace=[1], value=['Smile'])
dfv["gender"] = dfv["gender"].replace(to_replace=[-1], value=['Female'])
dfv["gender"] = dfv["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dfv["smiling"])
one_hot_g = pd.get_dummies(dfv["gender"])
dfv = dfv.drop(columns=["gender", "smiling"])
dfv = dfv.join(one_hot_g)
dfv = dfv.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(
    dataframe=dfv, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Frown", "Smile"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 999 validated image filenames.


In [60]:
model = Sequential()
model.add(InputLayer(input_shape=(55,45,1)))
model.add(Conv2D(filters=100, kernel_size=(3,3), strides=2, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=200, kernel_size=(2,2), strides=2, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=400,kernel_size=(2,2),activation="relu"))
model.add(Flatten())
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

In [61]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_77 (Conv2D)           (None, 27, 22, 100)       1000      
_________________________________________________________________
batch_normalization_23 (Batc (None, 27, 22, 100)       400       
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 13, 11, 100)       0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 6, 5, 200)         80200     
_________________________________________________________________
batch_normalization_24 (Batc (None, 6, 5, 200)         800       
_________________________________________________________________
max_pooling2d_64 (MaxPooling (None, 3, 2, 200)         0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 2, 1, 400)       

In [62]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [63]:
train_steps = train_generator.n//train_generator.batch_size
test_steps = test_generator.n//test_generator.batch_size

In [64]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=train_steps,
                    validation_data=test_generator,
                    validation_steps=test_steps,
                    epochs=50)

Epoch 1/50
40/40 [==============================] - 12s 294ms/step - loss: 0.7849 - accuracy: 0.6105 - val_loss: 0.6917 - val_accuracy: 0.4911
Epoch 2/50
40/40 [==============================] - 13s 323ms/step - loss: 0.5479 - accuracy: 0.7312 - val_loss: 0.6908 - val_accuracy: 0.5150
Epoch 3/50
40/40 [==============================] - 12s 298ms/step - loss: 0.4495 - accuracy: 0.7843 - val_loss: 0.7086 - val_accuracy: 0.5250
Epoch 4/50
40/40 [==============================] - 11s 275ms/step - loss: 0.3752 - accuracy: 0.8353 - val_loss: 0.9376 - val_accuracy: 0.5050
Epoch 5/50
40/40 [==============================] - 11s 281ms/step - loss: 0.3228 - accuracy: 0.8583 - val_loss: 1.0056 - val_accuracy: 0.5095
Epoch 6/50
40/40 [==============================] - 11s 272ms/step - loss: 0.2629 - accuracy: 0.8842 - val_loss: 0.9777 - val_accuracy: 0.5039
Epoch 7/50
40/40 [==============================] - 11s 270ms/step - loss: 0.2199 - accuracy: 0.9047 - val_loss: 1.3228 - val_accuracy: 0.5228